In [1]:
using Pipe
using Iterators
using HDF5, JLD

In [2]:
using PyCall
import PyCall.@pysym
@pyimport warnings
warnings.filterwarnings("ignore")

In [3]:
unshift!(PyVector(pyimport("sys")["path"]), "");

In [4]:
@pyimport nltk.corpus as nltk_corpus

In [5]:
#state = inner_inst, next
immutable PyObjectIter
    inner::PyObject 

    function PyObjectIter(obj::PyObject)
        inner = pycall(obj["__iter__"],PyObject)
        new(inner)
    end
end
function next_or_nothing(pyiter::PyObject)
    try
        pycall(pyiter["next"],PyObject)
    catch PyError
        nothing
    end
end

function Base.start(iter::PyObjectIter)
    inner_inst = deepcopy(iter.inner)
    next = next_or_nothing(inner_inst)
    inner_inst, next
end

function Base.next(pyiter::PyObjectIter, state::(PyObject,Union(Nothing,PyObject)))
    inner_inst, next = state
    new_next = next_or_nothing(inner_inst)
    next, (inner_inst,new_next)
end

function Base.done(pyiter::PyObjectIter, state::(PyObject,Union(Nothing,PyObject)))
    inner_inst, next = state
    next==nothing
end

done (generic function with 75 methods)

In [18]:
function cnf!(x::PyObject)
    x[:collapse_unary]()
    x[:chomsky_normal_form]()
    
    x
end

cnf! (generic function with 1 method)

In [32]:

function tupup(x::Any)
    x
end

function tupup(tree::PyObject)
    if pyisinstance(tree, @pysym :PyList_Type)
        if tree[:__len__]()==0
            nothing
        elseif tree[:__len__]()==1
            tupup(pyeval("tree[0]",PyObject; tree=tree))
        else
            @assert tree[:__len__]()==2
            left = tupup(pyeval("tree[0]",PyObject; tree=tree);)
            right = tupup(pyeval("tree[1]",PyObject; tree=tree);)
            if left!=nothing &&  right!=nothing
                (tupup(left), tupup(right))
            elseif left==nothing
                tupup(right)
            elseif right==nothing
                tupup(left)
            end
            
            
        end
    else
        convert(String,tree)
    end
end
    
function tupup(x::Nothing)
    error("Can't Make tuple of nothing")
end

tupup (generic function with 3 methods)

In [33]:
function loadtrees(reader::PyObject)
    @pipe (
    pycall(reader["parsed_sents"], PyObject) 
    |> PyObjectIter
    |> imap(cnf!,_)
    |> imap(tupup,_) 
    |> filter(x->x!==nothing,_) 
    |> collect)
end

loadtrees (generic function with 1 method)

In [12]:
open("training_sents.jsz","w") do fs
    serialize(fs, loadtrees(nltk_corpus.treebank))
end

In [36]:
@pyimport glob
opinosis_filenames = glob.glob("../Corpus/corpora/Opinosis/processed/topics/*.parsed.txt");
opinosis_reader = nltk_corpus.BracketParseCorpusReader(".", opinosis_filenames)

open("opinosis_sents.jsz","w") do fs
    serialize(fs, loadtrees(opinosis_reader))
end

Bad tree detected; trying to recover...
  Recovered by returning a flat parse.


LoadError: PyError (:PyObject_Call) <type 'exceptions.AttributeError'>
AttributeError("'tuple' object has no attribute 'label'",)
  File "/usr/local/lib/python2.7/dist-packages/nltk/tree.py", line 454, in chomsky_normal_form
    chomsky_normal_form(self, factor, horzMarkov, vertMarkov, childChar, parentChar)
  File "/usr/local/lib/python2.7/dist-packages/nltk/treetransforms.py", line 146, in chomsky_normal_form
    childNodes = [child.label() for child in node]

while loading In[36], in expression starting on line 5

In [35]:
questionbank_reader = nltk_corpus.BracketParseCorpusReader(".", ["../Corpus/corpora/QuestionBank/4000qs.txt"])

open("questionbank_sents.jsz","w") do fs
    serialize(fs, loadtrees(questionbank_reader))
end